# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [3]:
earthquake_json['metadata']

{'generated': 1732888198000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-10-29&endtime=2024-11-28',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.1',
 'count': 7719}

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 0.72,
  'place': '8 km WNW of Cobb, CA',
  'time': 1732751937530,
  'updated': 1732754370406,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc75091891',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc75091891&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 8,
  'net': 'nc',
  'code': '75091891',
  'ids': ',nc75091891,',
  'sources': ',nc,',
  'types': ',nearby-cities,origin,phase-data,scitech-link,',
  'nst': 7,
  'dmin': 0.0117,
  'rms': 0,
  'gap': 98,
  'magType': 'md',
  'type': 'earthquake',
  'title': 'M 0.7 - 8 km WNW of Cobb, CA'},
 'geometry': {'type': 'Point',
  'coordinates': [-122.814163208008, 38.8346672058105, 1.00999999046326]},
 'id': 'nc75091891'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [6]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,0.72,"8 km WNW of Cobb, CA",1732751937530,1732754370406,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc75091891,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",7.0,0.01170,0.0000,98.00,md,earthquake,"M 0.7 - 8 km WNW of Cobb, CA"
1,1.40,"17 km WNW of Tetlin, Alaska",1732751753405,1732751899997,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak024f9gg3g7,",",ak,",",origin,phase-data,",NaN,NaN,0.5300,NaN,ml,earthquake,"M 1.4 - 17 km WNW of Tetlin, Alaska"
2,1.50,"11 km WNW of Tatitlek, Alaska",1732751675692,1732751806997,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak024f9gfua9,",",ak,",",origin,phase-data,",NaN,NaN,0.4200,NaN,ml,earthquake,"M 1.5 - 11 km WNW of Tatitlek, Alaska"
3,0.90,"8 km NW of Indian Springs, Nevada",1732751591519,1732755645715,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nn00888045,",",nn,",",origin,phase-data,",34.0,0.19200,0.0949,143.07,ml,earthquake,"M 0.9 - 8 km NW of Indian Springs, Nevada"
4,1.10,"3 km NW of The Geysers, CA",1732751533340,1732764868661,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc75091886,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",21.0,0.00724,0.0300,78.00,md,earthquake,"M 1.1 - 3 km NW of The Geysers, CA"


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>